In [27]:
!pip install azure-ai-ml --upgrade


In [4]:
from azureml.core import Workspace, Dataset
from azureml.core.compute import ComputeTarget

# Connect to Azure ML Workspace
ws = Workspace.from_config()


In [18]:

# Define variables
endpoint_name = "svm-batch-ep"  # Replace with your endpoint name
input_data_path = "../data/prediction_data.csv"    # Replace with your input data file path
output_data_path = "../data/output"     # Location to save output
compute_target_name = "computeclusterml" # Replace with your compute cluster name

In [19]:

# Upload input data to a datastore
datastore = ws.get_default_datastore()
'''
datastore.upload_files(
    files=[input_data_path],
    target_path="batch_inputs/",
    overwrite=True,
)
'''
input_data = Dataset.File.from_files(path=(datastore, "../data/prediction_data.csv"))
datastore

Uploading an estimated of 1 files
Uploading ../data/prediction_data.csv
Uploaded ../data/prediction_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-62880345-ed91-473e-8c36-060e4852e575",
  "account_name": "dp100learning0966509990",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [10]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

# Set up Azure ML client
subscription_id = "9c15fe3b-278f-4875-b39a-d756406c9775"  # Replace with your Azure subscription ID
resource_group = "DP-100"    # Replace with your resource group name
workspace_name = "dp_100_learning"    # Replace with your Azure ML workspace name

# Authenticate and initialize MLClient
credential = DefaultAzureCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

# List all batch endpoints
batch_endpoints = ml_client.batch_endpoints.list()

# Print batch endpoint names
print("List of Batch Endpoints:")
for endpoint in batch_endpoints:
    print(f"Name: {endpoint.name}, Description: {endpoint.description}")


List of Batch Endpoints:
Name: svm-batch-ep, Description: None


In [12]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import BatchJob

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import BatchJob

# Azure ML workspace details
subscription_id = "9c15fe3b-278f-4875-b39a-d756406c9775"  # Replace with your Azure subscription ID
resource_group = "DP-100"    # Replace with your resource group name
workspace_name = "dp_100_learning"    # Replace with your Azure ML workspace name

# Authenticate and initialize MLClient
credential = DefaultAzureCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

# Define variables
endpoint_name = "svm-batch-ep"  # Replace with your batch endpoint name
input_data_path = "input_data.csv"    # Replace with your input data file path
datastore_name = "workspaceblobstore" # Replace with your datastore name (default: workspaceblobstore)

# Submit a batch job
job_name = "svm-batch-job"
job = ml_client.batch_jobs.create_or_update(
    BatchJob(
        name=job_name,
        endpoint_name=endpoint_name,
        input_data=input_data,
        output_data=Input(
            type="uri_folder",
            path=f"azureml://datastores/{datastore_name}/paths/batch_outputs/"
        ),
        compute="batch-cluster",  # Replace with the name of your compute cluster
    )
)

print(f"Batch job '{job_name}' submitted successfully!")



In [ ]:

# Monitor job status
print("Monitoring job status...")
while True:
    job_details = ml_client.jobs.get(name=job_name)
    print(f"Current Status: {job_details.status}")
    if job_details.status in ["Completed", "Failed", "Canceled"]:
        break

print(f"Job '{job_name}' completed with status: {job_details.status}")

# Download results if job succeeded
if job_details.status == "Completed":
    output_folder = "./batch_output"
    ml_client.jobs.download(name=job_name, output_name="batch_outputs", download_path=output_folder)
    print(f"Output downloaded to: {output_folder}")

In [28]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import BatchJob
from azure.ai.ml import Input

# Authenticate and initialize MLClient
credential = DefaultAzureCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

# Endpoint and input data
endpoint_name = "svm-batch-ep"  # Replace with your batch endpoint name

input_data = Input(
    type="uri_file",
    path="azureml://datastores/workspaceblobstore/paths/batch_inputs/prediction_data.csv"
)

# Submit a batch job
job_name = "batch-prediction-job"
job = ml_client.jobs.create_or_update(
    BatchJob(
        name=job_name,
        compute=compute_target_name,
        input_data=input_data,
        job_type="batch",
        output_data=Input(
            type="uri_folder",
            path="azureml://datastores/workspaceblobstore/paths/batch_inputs/"
        ),
        endpoint_name=endpoint_name,
    )
)

print(f"Batch job '{job_name}' submitted successfully!")


AttributeError: 'BatchJob' object has no attribute 'compute'